# Style-Bert-VITS2 (ver 2.4.1) のGoogle Colabでの学習

Google Colab上でStyle-Bert-VITS2の学習を行うことができます。

このnotebookでは、通常使用ではあなたのGoogle Driveにフォルダ`Style-Bert-VITS2`を作り、その内部での作業を行います。他のフォルダには触れません。
Google Driveを使わない場合は、初期設定のところで適切なパスを指定してください。

## 流れ

### 学習を最初からやりたいとき
上から順に実行していけばいいです。音声合成に必要なファイルはGoogle Driveの`Style-Bert-VITS2/model_assets/`に保存されます。また、途中経過も`Style-Bert-VITS2/Data/`に保存されるので、学習を中断したり、途中から再開することもできます。

### 学習を途中から再開したいとき
0と1を行い、3の前処理は飛ばして、4から始めてください。スタイル分け5は、学習が終わったら必要なら行ってください。


## 0. 環境構築

Style-Bert-VITS2の環境をcolab上に構築します。グラボモードが有効になっていることを確認し、以下のセルを順に実行してください。

**最近のcolabのアップデートにより、エラーダイアログ「WARNING: The following packages were previously imported in this runtime: [pydevd_plugins]」が出るが、「キャンセル」を選択して続行してください。**

In [ ]:
# このセルを実行して環境構築してください。
# エラーダイアログ「WARNING: The following packages were previously imported in this runtime: [pydevd_plugins]」が出るが「キャンセル」を選択して続行してください。

!git clone https://github.com/litagin02/Style-Bert-VITS2.git
%cd Style-Bert-VITS2/
!pip install -r requirements.txt
!python initialize.py --skip_jvnv

In [ ]:
# Google driveを使う方はこちらを実行してください。

from google.colab import drive
drive.mount("/content/drive")

## 1. 初期設定

学習とその結果を保存するディレクトリ名を指定します。
Google driveの場合はそのまま実行、カスタマイズしたい方は変更して実行してください。

In [1]:
# 学習に必要なファイルや途中経過が保存されるディレクトリ
dataset_root = "/content/drive/MyDrive/Style-Bert-VITS2/Data"

# 学習結果（音声合成に必要なファイルたち）が保存されるディレクトリ
assets_root = "/content/drive/MyDrive/Style-Bert-VITS2/model_assets"

import yaml


with open("configs/paths.yml", "w", encoding="utf-8") as f:
    yaml.dump({"dataset_root": dataset_root, "assets_root": assets_root}, f)

## 2. 学習に使うデータ準備

すでに音声ファイル（1ファイル2-12秒程度）とその書き起こしデータがある場合は2.2を、ない場合は2.1を実行してください。

### 2.1 音声ファイルからのデータセットの作成（ある人はスキップ可）

音声ファイル（1ファイル2-12秒程度）とその書き起こしのデータセットを持っていない方は、（日本語の）音声ファイルのみから以下の手順でデータセットを作成することができます。Google drive上の`Style-Bert-VITS2/inputs/`フォルダに音声ファイル（wavファイル形式、1ファイルでも複数ファイルでも可）を置いて、下を実行すると、データセットが作られ、自動的に正しい場所へ配置されます。

In [ ]:
# 元となる音声ファイル（wav形式）を入れるディレクトリ
input_dir = "/content/drive/MyDrive/Style-Bert-VITS2/inputs"
# モデル名（話者名）を入力
model_name = "your_model_name"

# こういうふうに書き起こして欲しいという例文（句読点の入れ方・笑い方や固有名詞等）
initial_prompt = "こんにちは。元気、ですかー？ふふっ、私は……ちゃんと元気だよ！"

!python slice.py -i {input_dir} --model_name {model_name}
!python transcribe.py --model_name {model_name} --initial_prompt {initial_prompt} --use_hf_whisper

成功したらそのまま3へ進んでください

### 2.2 音声ファイルと書き起こしデータがすでにある場合

指示に従って適切にデータセットを配置してください。

次のセルを実行して、学習データをいれるフォルダ（1で設定した`dataset_root`）を作成します。

In [5]:
import os

os.makedirs(dataset_root, exist_ok=True)

次に、学習に必要なデータを、Google driveに作成された`Style-Bert-VITS2/Data`フォルダに配置します。

まず音声データ（wavファイルで1ファイルが2-12秒程度の、長すぎず短すぎない発話のものをいくつか）と、書き起こしテキストを用意してください。wavファイル名やモデルの名前は空白を含まない半角で、wavファイルの拡張子は小文字`.wav`である必要があります。

書き起こしテキストは、次の形式で記述してください。
```
****.wav|{話者名}|{言語ID、ZHかJPかEN}|{書き起こしテキスト}
```

例：
```
wav_number1.wav|hanako|JP|こんにちは、聞こえて、いますか？
wav_next.wav|taro|JP|はい、聞こえています……。
english_teacher.wav|Mary|EN|How are you? I'm fine, thank you, and you?
...
```
日本語話者の単一話者データセットで構いません。

### データセットの配置

次にモデルの名前を適当に決めてください（空白を含まない半角英数字がよいです）。
そして、書き起こしファイルを`esd.list`という名前で保存し、またwavファイルも`raw`というフォルダを作成し、あなたのGoogle Driveの中の（上で自動的に作られるはずの）`Data`フォルダのなかに、次のように配置します。
```
├── Data
│   ├── {モデルの名前}
│   │   ├── esd.list
│   │   ├── raw
│   │   │   ├── ****.wav
│   │   │   ├── ****.wav
│   │   │   ├── ...
```

## 3. 学習の前処理

次に学習の前処理を行います。必要なパラメータをここで指定します。次のセルに設定等を入力して実行してください。「～～かどうか」は`True`もしくは`False`を指定してください。

In [6]:
# 上でつけたフォルダの名前`Data/{model_name}/`
model_name = "your_model_name"

# JP-Extra （日本語特化版）を使うかどうか。日本語の能力が向上する代わりに英語と中国語は使えなくなります。
use_jp_extra = True

# 学習のバッチサイズ。VRAMのはみ出具合に応じて調整してください。
batch_size = 4

# 学習のエポック数（データセットを合計何周するか）。
# 100で多すぎるほどかもしれませんが、もっと多くやると質が上がるのかもしれません。
epochs = 100

# 保存頻度。何ステップごとにモデルを保存するか。分からなければデフォルトのままで。
save_every_steps = 1000

# 音声ファイルの音量を正規化するかどうか
normalize = False

# 音声ファイルの開始・終了にある無音区間を削除するかどうか
trim = False

# 読みのエラーが出た場合にどうするか。
# "raise"ならテキスト前処理が終わったら中断、"skip"なら読めない行は学習に使わない、"use"なら無理やり使う
yomi_error = "skip"

上のセルが実行されたら、次のセルを実行して学習の前処理を行います。

In [ ]:
from gradio_tabs.train import preprocess_all

preprocess_all(
    model_name=model_name,
    batch_size=batch_size,
    epochs=epochs,
    save_every_steps=save_every_steps,
    num_processes=2,
    normalize=normalize,
    trim=trim,
    freeze_EN_bert=False,
    freeze_JP_bert=False,
    freeze_ZH_bert=False,
    freeze_style=False,
    freeze_decoder=False, # ここをTrueにするともしかしたら違う結果になるかもしれません。
    use_jp_extra=use_jp_extra,
    val_per_lang=0,
    log_interval=200,
    yomi_error=yomi_error
)

## 4. 学習

前処理が正常に終わったら、学習を行います。次のセルを実行すると学習が始まります。

学習の結果は、上で指定した`save_every_steps`の間隔で、Google Driveの中の`Style-Bert-VITS2/Data/{モデルの名前}/model_assets/`フォルダに保存されます。

このフォルダをダウンロードし、ローカルのStyle-Bert-VITS2の`model_assets`フォルダに上書きすれば、学習結果を使うことができます。

In [ ]:
# 上でつけたモデル名を入力。学習を途中からする場合はきちんとモデルが保存されているフォルダ名を入力。
model_name = "your_model_name"


import yaml
from gradio_tabs.train import get_path

dataset_path, _, _, _, config_path = get_path(model_name)

with open("default_config.yml", "r", encoding="utf-8") as f:
    yml_data = yaml.safe_load(f)
yml_data["model_name"] = model_name
with open("config.yml", "w", encoding="utf-8") as f:
    yaml.dump(yml_data, f, allow_unicode=True)

In [ ]:
# 日本語特化版を「使う」場合
!python train_ms_jp_extra.py --config {config_path} --model {dataset_path} --assets_root {assets_root}

In [ ]:
# 日本語特化版を「使わない」場合
!python train_ms.py --config {config_path} --model {dataset_path} --assets_root {assets_root}

In [ ]:
# 学習結果を試す・マージ・スタイル分けはこちらから
!python app.py --share